# Day 3: Binary Diagnostic
https://adventofcode.com/2021/day/3

In [2]:
// https://stackoverflow.com/a/35820003

use std::{
    fs::File,
    io::{self, BufRead, BufReader},
    path::Path,
};

fn lines_from_file(filename: impl AsRef<Path>) -> io::Result<Vec<String>> {
    BufReader::new(File::open(filename)?).lines().collect()
}

let lines = lines_from_file("input/day03.txt").unwrap();

Transform the lines to vectors of `char` to simplify analysing the digits.

In [3]:
let digits = lines[0].len();
let number_of_lines = lines.len();
let lines_as_chars: Vec<Vec<char>> = lines.iter().map(|line| line.chars().collect()).collect();

## Part 1

As a simple strategy for finding out if zeroes or ones are dominant, we assign the values 1 and -1 to the digits `'1'` and `'0'`, respectively, sum the values for a given digit index for all binary numbers, and check if the result is positive or negative.

In [4]:
// '1' -> +1
// '0' -> -1

fn char_value(c: char) -> i32 {
    (c.to_digit(10).unwrap() as i32) * 2 - 1
}

In [5]:
let count_1_0: Vec<i32> =
(0..digits).map(|i| 
    lines_as_chars.iter().map(move |line| char_value(line[i])).fold(0, |s, x| s + x)).collect();

In [6]:
let most_str = count_1_0.iter().map(|&n| if n > 0 { '1' } else { '0' }).collect();
most_str

"101111111101"

In [7]:
let least_str = count_1_0.iter().map(|&n| if n < 0 { '1' } else { '0' }).collect();
least_str

"010000000010"

In [8]:
let most = i32::from_str_radix(most_str.as_str(), 2).unwrap();
most

3069

In [9]:
let least = i32::from_str_radix(least_str.as_str(), 2).unwrap();
least

1026

In [10]:
most * least

3148794

## Part 1 (more elegant solution with `itertools`)
https://docs.rs/itertools/0.10.3/itertools/trait.Itertools.html

In [11]:
:dep itertools = "0.10.3"

In [12]:
use itertools::Itertools;

A more elegant solution is to sort the digits at a given index for all numbers, use `.group_by()` to group them by their values, and count the number of items in each group.

The return value of the function is a `Vec`, where the digits at indices 0 and 1 are the most common and least common values, respectively.

In [13]:
fn most_common_values_for_bit(numbers: &Vec<Vec<char>>, bit_index: usize) -> Vec<char> {
    numbers
        .iter()
        .map(|s| s[bit_index])
        .sorted()
        .group_by(|&c| c)
        .into_iter()
        .map(|(digit, group)| (digit, group.count()))
        .sorted_by_key(|(_digit, count)| *count)
        .rev()
        .map(|(digit, _group)| digit)
        .collect()
}

In [14]:
fn calculate_rate(numbers: &Vec<Vec<char>>, rank: usize) -> i32 {
    let digits = numbers[0].len();
    let rate_chars: String = (0..digits)
        .map(|bit_index| most_common_values_for_bit(numbers, bit_index)[rank])
        .collect();
    i32::from_str_radix(rate_chars.as_str(), 2).unwrap()
}

In [15]:
let gamma_rate = calculate_rate(&lines_as_chars, 0);
let epsilon_rate = calculate_rate(&lines_as_chars, 1);
gamma_rate * epsilon_rate

3148794

## Part 2

In part 2, things get a bit more complicated because numbers are removed successively, and it is not guaranteed that either `'0'` or `'1'` occurs strictly more often than the other digit. There could even be just a single digit left in the set of numbers under consideration.

Therefore, we have to consider the special cases that only one digit is present, or both are present, but occur equally often. In the latter case, we chose `'1'` and `'0'` for the most and least common digit, respectively.

In [16]:
#[derive(Copy, Clone)]
enum ValueSelector {
    MostCommon,
    LeastCommon
}

fn value_for_bit(numbers: &Vec<&Vec<char>>, bit_index: usize, value_selector: ValueSelector) -> char {
    let values_sorted_by_frequency: Vec<(char, usize)> = numbers
        .iter()
        .map(|s| (**s)[bit_index])
        .sorted()
        .group_by(|&c| c)
        .into_iter()
        .map(|(digit, group)| (digit, group.count()))
        .sorted_by_key(|(_digit, count)| *count)
        .collect();
    
    match values_sorted_by_frequency.len() {
        1 => values_sorted_by_frequency[0].0,
        2 => if values_sorted_by_frequency[0] == values_sorted_by_frequency[1] {
                match value_selector {
                    // special case: both values equally common
                    ValueSelector::LeastCommon => '0',
                    ValueSelector::MostCommon => '1'
                }
            } else {
                values_sorted_by_frequency[match value_selector {
                    ValueSelector::LeastCommon => 0,
                    ValueSelector::MostCommon => 1
                }].0
            }
        _ => panic!("Expected 1 or 2 items in values_sorted_by_frequency.")
    }
}

The next function takes a list of numbers, and filters them according to the given bit criteria.

In [17]:
fn filter_bit_criteria<'a>(numbers: &Vec<&'a Vec<char>>,
                           bit_index: usize,
                           value_selector: ValueSelector) -> Vec<&'a Vec<char>> {
    let filter_value = value_for_bit(numbers, bit_index, value_selector);
    numbers
        .iter()
        .filter(|number| number[bit_index] == filter_value)
        .map(|&n| n)
        .collect()
}

Finally, `calculate_rating(numbers, value_selector)` applies `filter_bit_criteria()` repeatedly for each bit index to the given numbers until only one number remains.

In [18]:
fn calculate_rating(numbers: &Vec<Vec<char>>, value_selector: ValueSelector) -> i32 {
    let digits = numbers[0].len();
    let numbers_ref: Vec<&Vec<char>> = numbers.iter().collect();
    let rate_string: String = (0..digits)
        .scan(numbers_ref, |remaining, bit_index| {
            *remaining = filter_bit_criteria(remaining, bit_index, value_selector);
            Some(remaining.to_owned())
        })
        .skip_while(|remaining| remaining.len() > 1)
        .next()
        .unwrap()  // Vec<&Vec<char>>
        .iter()
        .next()
        .unwrap()  // Vec<char>
        .iter()
        .collect();
    i32::from_str_radix(rate_string.as_str(), 2).unwrap()
}

In [19]:
let oxygen_rating = calculate_rating(&lines_as_chars, ValueSelector::MostCommon);
let co2_rating = calculate_rating(&lines_as_chars, ValueSelector::LeastCommon);
oxygen_rating * co2_rating

2795310